In [1]:
import time
import pandas as pd
from wikiapi import WikiApi
wiki = WikiApi()
import re
from gensim.models import Word2Vec
#import word2vec
from scipy.spatial.distance import cosine
import numpy as np
import heapq

In [3]:
def get_longword(s):
    return heapq.nlargest(2, s, key=len)

In [4]:
#get wiki for longword later

In [156]:


def get_avg_vec(words):
	try:
		words = re.split(' ', words)
	except:
		pass
	nwords = 0
	featureVec = np.zeros((num_features,))#,dtype="float32")
	for word in words:
		if word in index2word_set:
			nwords = nwords + 1.
			featureVec = np.add(featureVec,model[word])
	if nwords > 0:
		featureVec = np.divide(featureVec,nwords)
	return featureVec

In [7]:
def get_tvec(row):
    if row['correctAnswer'] == 'A':
        a = row['aavec']
        return a
    elif row['correctAnswer'] == 'B':
        a = row['bavec']
        return a
    elif row['correctAnswer'] == 'C':
        a = row['cavec']
        return a
    elif row['correctAnswer'] == 'D':
        a = row['davec']
        return a

In [10]:
start = time.time()
model = Word2Vec.load_word2vec_format('/Users/liamconnell/Downloads/GoogleNews-vectors-negative300.bin', binary = True)
lap1 = time.time()
print('data gathered: %s' % (lap1 - start))

data gathered: 85.35520815849304


In [11]:
index2word_set = set(model.index2word)
num_features = 300

In [12]:
data  = pd.read_csv('../input/training_set.tsv', '\t')

In [13]:
data['keyword'] =data['question'].apply(get_longword)

In [157]:
data['qvec'] = data.question.apply(get_avg_vec) 

In [158]:
#data['qvec'] = data.keyword.apply(get_avg_vec)              #later change kwords to words

data['aavec'] = data['answerA'].apply(get_avg_vec)
data['bavec'] = data['answerB'].apply(get_avg_vec)
data['cavec'] = data['answerC'].apply(get_avg_vec)
data['davec'] = data['answerD'].apply(get_avg_vec)

#data['tvec'] = data.apply(get_tvec, axis = 1)

In [165]:
tvec = []
for i in range(len(data)):
    tvec.append(get_tvec(data.iloc[i,:]))

In [166]:
len(tvec)

2500

In [167]:
data['tvec'] = tvec

In [65]:
theta = np.identity(300)
alpha = .001

In [66]:
row = data.iloc[0]
cosine(row['tvec'].dot(theta), row['qvec'])

0.87289437680068405

In [234]:
def cost_function(data, theta):
    dists =  [cosine(data.iloc[i]['qvec'].dot(theta), data.iloc[i]['tvec']) for i in data.index]
    return np.nansum(dists)/len(data)          #np.nansum

In [262]:
def update_theta(data, theta, alpha):
    x = np.array([data.iloc[i]['qvec'] for i in data.index])    
    xTrans = x.transpose()
    ttheta = theta.copy()
    for m in range(300):
        #print(m)
        y = np.array([data.iloc[i]['tvec'][0] for i in data.index])
        hypothesis = np.dot(x, theta[m,])
        loss = hypothesis - y
        gradient = np.dot(xTrans, loss) / len(data)  
        ttheta[m,] = ttheta[m,] - alpha * gradient  
        
    return ttheta

In [263]:
def gradient_descent(data,theta, alpha = .1, iter = 4):
    #theta = np.identity(300)
    cost_hist = []
    cost = cost_function(data, theta)
    print(cost)
    cost_hist.append(cost)
    for i in range(iter):
        alpha_1 = alpha#/(i+1)
        theta = update_theta(data, theta, alpha_1)
        cost = cost_function(data, theta)
        print(cost)
        cost_hist.append(cost)
    print(cost_hist)
    return theta

In [223]:
theta = np.identity(300)
alpha = .001
#theta = update_theta(data, theta, alpha)

In [252]:
theta = np.identity(300)
theta = gradient_descent(data, iter = 4)

0.52683392424
0.526834974887
0.526836025555
0.526837076243
0.526838126953
[0.52683392424012054, 0.52683497488705366, 0.52683602555481079, 0.52683707624339227, 0.52683812695279775]


In [254]:
theta = np.identity(300)
theta = gradient_descent(data,alpha = .1, iter = 4)

0.52683392424
0.527902589475
0.528992395891
0.530103352252
0.531235454813
[0.52683392424012054, 0.52790258947456536, 0.52899239589097835, 0.53010335225187533, 0.53123545481267698]


In [257]:
theta = np.identity(300)
theta = gradient_descent(data,alpha = 10, iter = 4)

0.52683392424
0.495587009925
0.495912546395
0.49846739793
0.500625576535
[0.52683392424012054, 0.49558700992520988, 0.49591254639506144, 0.49846739792995592, 0.50062557653508288]


In [259]:
theta = np.identity(300)
theta = gradient_descent(data,alpha = 10, iter = 4)

0.52683392424
0.495587009925
0.498168580749
0.503878122274
0.507196279844
[0.52683392424012054, 0.49558700992520988, 0.49816858074917308, 0.5038781222737394, 0.50719627984359927]


In [261]:
theta = np.identity(300)
theta = gradient_descent(data,theta,alpha = .001, iter = 4)

0.52683392424
0.52682341973
0.526812917302
0.526802416956
0.526791918692
[0.52683392424012054, 0.52682341973008295, 0.52681291730210655, 0.52680241695616437, 0.52679191869222963]


In [264]:
theta = np.identity(300) * -1
theta = gradient_descent(data,theta,alpha = 1, iter = 100)

1.40356607576
1.39095608591
1.37595782796
1.3587957342
1.33986459422
[1.4035660757598794, 1.3909560859096859, 1.3759578279619258, 1.358795734201631, 1.3398645942247911]


In [268]:
theta = np.random.rand(300,300)
theta = gradient_descent(data,theta,alpha = 1, iter = 100)

0.950599208129
0.950382562211
0.950203245672
0.950056424337
0.949937788749
0.949843535315
0.94977033077
0.949715268973
0.949675825574
0.949649813762
0.949635342844
0.94963078048
0.949634718834
0.949645944621
0.949663412787
0.949686223549
0.949713602422
0.949744882914
0.949779491562
0.949816935002
0.949856788826
0.94989868797
0.949942318439
0.94998741018
0.950033730947
0.950081081024
0.95012928868
0.950178206262
0.950227706829
0.950277681261
0.950328035771
0.95037868976
0.950429573978
0.950480628935
0.950531803542
0.950583053936
0.95063434247
0.950685636846
0.950736909365
0.950788136289
0.950839297285
0.95089037495
0.950941354401
0.950992222926
0.951042969675
0.951093585405
0.951144062255
0.951194393548
0.951244573633
0.951294597736
0.951344461837
0.95139416257
0.951443697125
0.951493063177
0.951542258811
0.951591282469
0.951640132899
0.951688809112
0.951737310347
0.951785636034
0.951833785771
0.951881759302
0.951929556492
0.951977177311
0.952024621823
0.952071890167
0.952118982552
0.95

In [250]:
theta = np.identity(300)
theta = gradient_descent(data, iter = 4)

0.52683392424
0.52693916551
0.527044615337
0.527150273748
0.527256140765
[0.52683392424012054, 0.52693916550979336, 0.52704461533741265, 0.52715027374785117, 0.52725614076479965]


In [248]:
cosine([1,0],[-2,-2])

1.7071067811865475

In [230]:
cost_hist

NameError: name 'cost_hist' is not defined

In [106]:
def update_theta(data, theta, alpha):
    for m in range(300):
        for n in range(300):
            cost = [(data.iloc[i]['qvec'].dot(theta) - data.iloc[i]['tvec']) * data.iloc[i]['qvec'] for i in data.index]
            theta[m,n] = theta[m,n] - alpha/len(data) * np.nansum(cost)
    return theta

In [118]:
def update_theta(data, theta, alpha):
    for m in range(300):
        print(m)
        for n in range(300):
            #print(n)
            diff = [np.array(data.iloc[i]['qvec']*theta[m,] - data.iloc[i]['tvec'][n])[m] * data.iloc[i]['qvec'][m] 
                    for i in data.index]
            #sel = [data.iloc[i]['qvec'][m] for i in data.index]                
            #grad = [a[m]*b for a,b in zip(diff,sel)]
            theta[m,n] = theta[m,n] - alpha/len(data) * np.nansum(diff)
    return theta

In [97]:
cost_function(data, theta)

0.85797213426555552

In [146]:
x = np.array([data.iloc[i]['qvec'] for i in data.index])    
xTrans = x.transpose()

In [200]:
count = 0
for i in data.index:
    if sum(data.iloc[i]['tvec']) == 0:
        count = count + 1
        print(data.iloc[i])
count

id                                                          100042
question         Which type of organism is at the lowest level ...
correctAnswer                                                    A
answerA                                                  autotroph
answerB                                                heterotroph
answerC                                                  carnivore
answerD                                                   omnivore
keyword                                                     [W, h]
qvec             [-0.00111744366586, 0.0104076072224, 0.0331170...
aavec            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
bavec            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
cavec            [-0.00819526519626, 0.0370802171528, -0.035199...
davec            [0.0611092448235, -0.00446789292619, -0.064856...
tvec             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: 41, dtype: object
id                                    

87

In [44]:
#data['tvec'] = 0
#data['tvec'] =  data.apply(get_tvec, axis = 1)

for i in range(len(data)):
    tvec.append(get_tvec(data.iloc[i,:]))

100
500
1000
2000


In [39]:
data.iloc[0, 'tvec'] = get_tvec(data.iloc[0,:])

ValueError: Can only index by location with a [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array]

In [37]:

data.loc[0, 'tvec']=get_tvec(data.loc[0])

ValueError: Must have equal len keys and value when setting with an iterable

In [18]:
row = data.loc[0]
row['davec'] = 0

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [22]:
data.loc[0,'tvec'] = 0

In [23]:
data.head()

,id,question,correctAnswer,answerA,answerB,answerC,answerD,keyword,qvec,aavec,bavec,cavec,davec,tvec
0,0,"When athletes begin to exercise, their heart r...",C,at the tissue level,at the organ level,at the system level,at the cellular level,"[W, h]","[-0.107751, 0.0529086, -0.021548, 0.0340352, -...","[-0.0238058, 0.0379852, 0.0209473, 0.0392717, ...","[0.00312668, 0.0361639, 0.0316554, 0.0376649, ...","[-0.00687617, 0.00266938, 0.0359293, 0.0547588...","[-0.0181546, 0.00685427, 0.020112, 0.0437048, ...",0
1,100002,Which example describes a learned behavior in ...,C,smelling the air for odors,barking when disturbed,sitting on command,digging in soil,"[W, h]","[-0.107751, 0.0529086, -0.021548, 0.0340352, -...","[0.00602225, 0.0450306, 0.0368578, -0.0060186,...","[0.0266252, 0.0278478, 0.0564013, -0.0153443, ...","[0.0628858, 0.0293218, -0.000488449, 0.0688045...","[0.000976153, 0.102385, 0.057852, 0.0386183, 0...",NaN
2,100003,"When two nuclei are combined into one nucleus,...",D,conversion,reaction,fission,fusion,"[W, h]","[-0.107751, 0.0529086, -0.021548, 0.0340352, -...","[0.0507117, 0.0424134, -0.0399547, -0.0341152,...","[0.0267808, -0.0442315, 0.0214246, -0.00150102...","[-0.0163885, 0.110206, 0.0577159, 0.0612786, -...","[-0.0258577, 0.000564112, 0.110586, 0.0546427,...",NaN
3,100004,Which is a distinction between an epidemic and...,B,the symptoms of the disease,the geographical area affected,the species of organisms infected,the season in which the disease spreads,"[W, h]","[-0.107751, 0.0529086, -0.021548, 0.0340352, -...","[0.0318869, 0.0791147, 0.0059272, 0.0598625, -...","[0.00613054, 0.0375127, 0.0308274, 0.0274943, ...","[0.0524637, 0.0690872, 0.0192883, 0.034781, -0...","[0.0286547, 0.068451, 0.0140399, 0.0502198, -0...",NaN
4,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...,"[I, n]","[0.013328, -0.00751545, 0.0104983, 0.086481, -...","[-0.0176523, 0.0291838, 0.00984529, 0.0267574,...","[-0.0155491, 0.0247668, 0.00519347, 0.0275544,...","[-0.0177486, 0.0154555, 0.001331, 0.0448149, -...","[-0.0164198, 0.00993515, 0.000937077, 0.039094...",NaN
